In [3]:
import contextlib, traceback

@contextlib.contextmanager
def trace_eager():
    patches = []
    def patch(obj, attr, make_wrapper):
        orig = getattr(obj, attr)
        wrapper = make_wrapper(orig)
        setattr(obj, attr, wrapper)
        patches.append((obj, attr, orig))
    def stack(msg):
        print(f"\n=== EAGER TRIGGER: {msg} ===")
        traceback.print_stack(limit=80)

    import dask.base as _db
    patch(_db, "compute", lambda orig: (lambda *a, **k: (stack("dask.base.compute"), orig(*a, **k))[-1]))

    try:
        from distributed import Client as _Client
        patch(_Client, "compute", lambda orig: (lambda self,*a,**k: (stack("distributed.Client.compute"), orig(self,*a,**k))[-1]))
        if hasattr(_Client, "get"):
            patch(_Client, "get", lambda orig: (lambda self,dsk,keys,*a,**k: (stack("distributed.Client.get (graph submit)"), orig(self,dsk,keys,*a,**k))[-1]))
    except Exception:
        pass

    import dask.array as da
    patch(da.core.Array, "__array__", lambda orig: (lambda self, dtype=None: (stack("dask.array.Array.__array__"), orig(self, dtype=dtype))[-1]))

    import xarray as xr
    patch(xr.Dataset, "compute", lambda orig: (lambda self, **k: (stack("xarray.Dataset.compute"), orig(self, **k))[-1]))
    patch(xr.DataArray, "compute", lambda orig: (lambda self, **k: (stack("xarray.DataArray.compute"), orig(self, **k))[-1]))
    patch(xr.Dataset, "load",    lambda orig: (lambda self, **k: (stack("xarray.Dataset.load"),    orig(self, **k))[-1]))
    patch(xr.DataArray, "load",  lambda orig: (lambda self, **k: (stack("xarray.DataArray.load"),  orig(self, **k))[-1]))

    patch(da, "store", lambda orig: (lambda *a, **k: (stack("dask.array.store"), orig(*a, **k))[-1]))

    try:
        yield
    finally:
        for obj, attr, orig in reversed(patches):
            setattr(obj, attr, orig)


In [2]:
from dask.distributed import Client, get_task_stream
import dask, xarray as xr

client = Client()
dask.config.set({"optimization.fuse.active": False})  # clearer task names

href = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:202504-s02msil2a/15/products/cpm_v256/S2A_MSIL2A_20250415T094041_N0511_R036_T35VLC_20250415T155631.zarr"

with trace_eager(), get_task_stream(client=client) as ts:
    ds = xr.open_dataset(href, engine="eopf-zarr", chunks={}, variables=["b02","b03","b04"])

print("Tasks during open:", len(ts.data))



=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/opt/miniconda3/envs/eopf-example/lib/pyth


=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/opt/miniconda3/envs/eopf-example/lib/pyth


=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===

=== EAGER TRIGGER: dask.base.compute ===

=== EAGER TRIGGER: distributed.Client.get (graph submit) ===
Tasks during open: 52


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/opt/miniconda3/envs/eopf-example/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/opt/miniconda3/envs/eopf-example/lib/pyth